In [2]:
%load_ext autoreload
%autoreload 2
from alphatoe import evals, data, game
import torch
from torch.nn.functional import cross_entropy
from tqdm import tqdm
from transformer_lens import HookedTransformerConfig, HookedTransformer
import json

In [28]:
#generate games
_, game_list = data.gen_games(gametype="all")

Generating all possible games...
Generated 255168 games
Generated array of moves


In [46]:
# fine tuned on strat after 1000 all
# model_dict = torch.load("../scripts/models/fine-tuning 4layer-20230713-155846.pt")
# 4k iter strat games
model_file = "../scripts/models/strat 4layer-20230713-161937"
model_dict = torch.load(f"{model_file}.pt")
with open(f"{model_file}.json") as json_file:
    cfg_dict = json.load(json_file)

cfg_dict

{'experiment_name': 'strat 4layer',
 'gametype': 'strat',
 'fine_tune': 'recent',
 'n_epochs': 1000,
 'lr': 1e-05,
 'weight_decay': 0.0001,
 'batch_size': 4096,
 'train_test_split': 0.8,
 'n_layers': 4,
 'n_heads': 8,
 'd_model': 128,
 'd_head': 16,
 'd_mlp': 512,
 'act_fn': 'relu',
 'normalization_type': None,
 'device': 'cuda',
 'seed': 1337,
 'save_losses': False,
 'save_checkpoints': False,
 'eval_model': False,
 'd_vocab': 11,
 'd_vocab_out': 10,
 'n_ctx': 10,
 'init_weights': True}

In [44]:
#load json config as dictionary
with open("../scripts/models/games-20230711-195141.json") as json_file:
    cfg_dict = json.load(json_file)

cfg_dict

{'experiment_name': 'strategic games',
 'gametype': 'strat',
 'fine_tune': None,
 'n_epochs': 500,
 'lr': 1e-05,
 'weight_decay': 0.0001,
 'batch_size': 32768,
 'train_test_split': 0.8,
 'n_layers': 8,
 'n_heads': 8,
 'd_model': 128,
 'd_head': 16,
 'd_mlp': 512,
 'act_fn': 'relu',
 'normalization_type': None,
 'device': 'cuda',
 'seed': 1337,
 'save_losses': False,
 'save_checkpoints': False,
 'eval_model': False}

In [47]:
cfg = HookedTransformerConfig(
    n_layers = cfg_dict['n_layers'],
    n_heads = cfg_dict['n_heads'],
    d_model = cfg_dict['d_model'],
    d_head = cfg_dict['d_head'],
    d_mlp = cfg_dict['d_mlp'],
    act_fn = cfg_dict['act_fn'],
    #normalization_type=None,
    normalization_type=cfg_dict['normalization_type'],
    d_vocab=11,
    d_vocab_out=10,
    n_ctx=10,
    init_weights=True,
    device="cuda",
    seed = 1337,
)

model = HookedTransformer(cfg)
model.load_state_dict(model_dict)

<All keys matched successfully>

In [8]:
model.to("cuda")

Moving model to device:  cuda


HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): Identity()
      (ln2): Identity()
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_mid): HookPoint()
      (hook_resid_post): HookPoint()
    )
    (1): TransformerBlock(
      (ln1): Identity()
      (ln2): Identity()
      (attn): Attention(
        (hook_k): HookPoint()
      

In [48]:
seq_test = [10,3,1,7]
model(torch.tensor(seq_test))[0, -1]

tensor([  9.3121, -11.4234,   2.7156, -21.8320,   6.9190,   7.6798,  10.4413,
        -18.3917,   8.9384,  -2.6937], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [49]:
sample = evals.sample_games(model, 1, 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:14<00:00, 68.84it/s]


In [50]:
len(sample[11])

11

In [51]:
eval_counts, eval_games = evals.eval_model(sample)

0it [00:00, ?it/s]

1000it [00:00, 13490.63it/s]


In [52]:
eval_counts

{'_check_played_repeat_moves': 0.002,
 '_check_played_after_player_victory': 0.022,
 '_check_played_after_draw_game': 0.0,
 'inappropriate_end_state': 0.0,
 '_check_if_illegal_moves': 0.024}

In [42]:
game.play_game(sample[100])

| O | X | O |
| O | X | X |
| X | O | X |


In [ ]:
move = model(torch.tensor(seq))[0,-1]

In [14]:
game.play_game(sample[11])
print(sample[11])
print(evals._check_if_illegal_moves(sample[11]))

| O | O | X |
| X | X | O |
| O | X | X |
[10, 3, 6, 7, 5, 2, 0, 4, 1, 8, 9]
False


In [15]:
other_illegal_games = evals.eval_model(sample)

In [20]:
len(sample)
len(other_illegal_games)

2

In [18]:
bad_samples = [sample[i] for i in range(len(sample)) if other_illegal_games[i]]

IndexError: tuple index out of range

In [ ]:
game.play_game(bad_samples[0])
print(bad_samples[0])


Invalid game
| O | O | X |
| X | O | X |
|   | X | O |
[10, 2, 0, 3, 4, 5, 1, 7, 8, 9, 9]


In [23]:
game_overlap = [sample[i] for i in range(len(sample)) if (eval_games[i]["_check_played_repeat_moves"] and eval_games[i]["_check_played_after_player_victory"])]

In [24]:
overlap = len(game_overlap) / len(sample)

In [25]:
sum([eval_counts[eval] for eval in eval_counts if eval != "_check_if_illegal_moves"]) - overlap

0.027